In [1]:
import os
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)

In [2]:
DEMO_PATH = "notebook/integration_demo/"
ASKENET_PATH_1 = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"
ASKENET_PATH_2 = "test/models/AMR_examples/SIDARTHE.amr.json"

ASKENET_PATHS = [ASKENET_PATH_1, ASKENET_PATH_2]

In [3]:
def solution_mapping1(model1_solution: dict) -> dict:
    return model1_solution


def solution_mapping2(model2_solution: dict) -> dict:
    mapped_solution = {}
    mapped_solution["Susceptible"] = (
        model2_solution["Susceptible"]
        + model2_solution["Recognized"]
        + model2_solution["Threatened"]
    )

    mapped_solution["Infected"] = (
        model2_solution["Infected"]
        + model2_solution["Ailing"]
        + model2_solution["Diagnosed"]
    )

    # Model 1 doesn't include dead people, and implicitly assumes that everyone who is infected will recover.
    mapped_solution["Recovered"] = (
        model2_solution["Healed"] + model2_solution["Extinct"]
    )

    return mapped_solution


solution_mappings = [solution_mapping1, solution_mapping2]

## load_and_sample_ensemble_model

In [4]:
weights = [0.5, 0.5]
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling
ensemble_samples = load_and_sample_petri_ensemble(
    ASKENET_PATHS, weights, solution_mappings, num_samples, timepoints
)

# Save results
ensemble_samples.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/sample_results.csv"), index=False)

## load_and_calibrate_and_sample_ensemble_model

In [7]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_ensemble_model(
    ASKENET_PATHS,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=1000,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 56.12164771556854
iteration 25: loss = 49.60371994972229
iteration 50: loss = 33.30795359611511
iteration 75: loss = 20.579649567604065
iteration 100: loss = 21.66509783267975
iteration 125: loss = 18.753786206245422
iteration 150: loss = 16.53753125667572
iteration 175: loss = 17.371854722499847
iteration 200: loss = 15.905078649520874
iteration 225: loss = 13.789868116378784
iteration 250: loss = 13.533606469631195
iteration 275: loss = 14.916905522346497
iteration 300: loss = 13.741706490516663
iteration 325: loss = 14.56143856048584
iteration 350: loss = 12.594428539276123
iteration 375: loss = 14.353631019592285
iteration 400: loss = 14.124584197998047
iteration 425: loss = 16.33763426542282
iteration 450: loss = 15.56375253200531
iteration 475: loss = 15.068402886390686
iteration 500: loss = 14.676460444927216
iteration 525: loss = 15.804013967514038
iteration 550: loss = 14.724295616149902
iteration 575: loss = 14.85770708322525
iteration 600: loss = 14.05884